In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log
33,application_1607949680860_0035,pyspark,idle,Link,Link


SparkSession available as 'spark'.

In [2]:
import hashlib
from datetime import datetime
from pyspark.sql import functions as func
from pyspark.sql.types import FloatType
import hsfs

In [3]:
# Create a connection
connection = hsfs.connection()
# Get the feature store handle for the project's feature store
fs = connection.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

## Placeholder

In [4]:
transactions_fg = fs.get_feature_group("transactions_fg", 1)

In [5]:
transactions_df = transactions_fg.read()
transactions_df.show()

+------+--------+------+--------+------+-------+--------------+-------+
|is_sar|base_amt|target|alert_id|source|tran_id|tran_timestamp|tx_type|
+------+--------+------+--------+------+-------+--------------+-------+
|     0| 9405.71|  3259|      -1|  1767|      1|   1.4832288E9|      4|
|     0| 6884.54|  5141|      -1|  7363|      2|   1.4832288E9|      4|
|     0|  7968.4|  9532|      -1|  7585|      3|   1.4832288E9|      4|
|     0| 9042.67|  8792|      -1|  1750|      4|   1.4832288E9|      4|
|     0| 4692.79|  4670|      -1|  9060|      5|   1.4832288E9|      4|
|     0| 4089.65|  3861|      -1|  8752|      6|   1.4832288E9|      4|
|     0| 3055.04|  3805|      -1|  9645|      7|   1.4832288E9|      4|
|     0| 6473.45|  9280|      -1|  7150|      8|   1.4832288E9|      4|
|     0| 2583.42|  6369|      -1|  1685|      9|   1.4832288E9|      4|
|     0| 6753.04|  8255|      -1|  7846|     10|   1.4832288E9|      4|
|     0| 1743.57|  5957|      -1|   878|     11|   1.4832288E9| 

## Placeholder

In [6]:
edges_td_meta = fs.create_training_dataset(name="edges_td",
                                       version=1,
                                       data_format="csv",
                                       label = ["is_sar"],   
                                       description="edges training dataset")
edges_td_meta.save(transactions_df)

### Placeholder

In [7]:
account_fg = fs.get_feature_group("account_features", 1)

In [8]:
accounts_df = account_fg.read()
accounts_df.show()

+-------+---------------+--------------+---------+
|acct_id|initial_deposit|tx_behavior_id|prior_sar|
+-------+---------------+--------------+---------+
|      0|       84442.19|             1|        0|
|      1|       75795.44|             1|        0|
|      2|       42057.16|             1|        0|
|      3|       25891.68|             1|        0|
|      4|       51127.47|             1|        0|
|      5|       40493.41|             1|        0|
|      6|       78379.86|             1|        0|
|      7|       30331.27|             1|        0|
|      8|        47659.7|             1|        0|
|      9|        58338.2|             1|        0|
|     10|       90811.29|             1|        0|
|     11|       50468.69|             1|        0|
|     12|       28183.78|             1|        0|
|     13|       75580.42|             1|        0|
|     14|        61836.9|             1|        0|
|     15|       25050.63|             1|        0|
|     16|       90974.63|      

### Placeholder

In [9]:
source = transactions_fg.select(["source","is_sar"]).read().withColumnRenamed("source","id")
target = transactions_fg.select(["target","is_sar"]).read().withColumnRenamed("target","id")
nodes = source.union(target)
sar_nodes = nodes.where(nodes.is_sar == 1)
non_sar = nodes.where(nodes.is_sar == 0)
non_sar_nodes = non_sar.join(sar_nodes, ["id"], "leftanti")
labels = sar_nodes.union(non_sar_nodes).dropDuplicates(subset=["id"])
labels.show()

+----+------+
|  id|is_sar|
+----+------+
|3997|     1|
|5300|     1|
| 463|     1|
|8086|     1|
|7833|     1|
|1088|     0|
|6654|     0|
|9465|     0|
|7880|     0|
|6336|     0|
|1580|     0|
|1829|     0|
|7993|     0|
|7240|     0|
|7253|     0|
|8389|     0|
|4935|     0|
|6658|     0|
|6466|     0|
|8592|     0|
+----+------+
only showing top 20 rows

In [10]:
accounts_df = accounts_df.withColumnRenamed("acct_id","id")
accounts_df.show()

+---+---------------+--------------+---------+
| id|initial_deposit|tx_behavior_id|prior_sar|
+---+---------------+--------------+---------+
|  0|       84442.19|             1|        0|
|  1|       75795.44|             1|        0|
|  2|       42057.16|             1|        0|
|  3|       25891.68|             1|        0|
|  4|       51127.47|             1|        0|
|  5|       40493.41|             1|        0|
|  6|       78379.86|             1|        0|
|  7|       30331.27|             1|        0|
|  8|        47659.7|             1|        0|
|  9|        58338.2|             1|        0|
| 10|       90811.29|             1|        0|
| 11|       50468.69|             1|        0|
| 12|       28183.78|             1|        0|
| 13|       75580.42|             1|        0|
| 14|        61836.9|             1|        0|
| 15|       25050.63|             1|        0|
| 16|       90974.63|             1|        0|
| 17|       98278.55|             1|        0|
| 18|       8

In [11]:
accounts_df.count()

10000

In [12]:
nodes_df = accounts_df.join(labels, ["id"])
nodes_df.count()

9998

In [13]:
node_td = fs.create_training_dataset(name="node_td",
                                       version=1,
                                       data_format="csv",   
                                       description="node training dataset")
node_td.save(nodes_df)